In [ ]:
"""
COSC 525 - Deep Learning
Project #3: Building Networks with Tensorflow and Keras
Contributors: Anna-Maria Nau and Christoph Metzner
Date: 03/10/20
"""
# Import main libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from sklearn.metrics import confusion_matrix

# Import other libraries
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.utils import to_categorical

# Import Fashion-Mnist Dataset from Keras and Preprocess Dataset

In [ ]:
# Import fashion-mnist dataset from
# https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles
# Downloads the data, will take some time so make yourself some tea!
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# Change format of label dataset --> from 1 to [0,1,0,0,0,0,0,0,0,0]
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# from https://www.tensorflow.org/tutorials/keras/classification
# creat list with all respective class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Get min, and max value from training data set using all 60,000 samples
max_val = x_train.max()
print("Maximal value: ", max_val)
min_val = x_train.min()
print("Minimum value: ", min_val)


In [ ]:
# Init a normalization funciton
def normalization(matrix, max_val, min_val):
    new_matrix = np.array([((image - min_val)/(max_val-min_val)) for image in matrix])
    return new_matrix

In [ ]:
# Normalize training (x_train) and testing (x_test) datasets based on maximal and minimal values of the training dataset
x_train_scaled = normalization(x_train, max_val, min_val)
x_test_scaled = normalization(x_test, max_val, min_val)

# Show normalized image matrix and original matrix
print(x_train_scaled[10])
print(x_train[10])
print(x_train.shape)
print(x_train_scaled.shape)

In [ ]:
reshaped_x_train_scaled = np.reshape(x_train_scaled,(60000,28,28,1))
reshaped_x_test_scaled = np.reshape(x_test_scaled,(10000,28,28, 1))

In [ ]:
# from https://www.tensorflow.org/tutorials/keras/classification

# Plot first image of the training dataset
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()
##########

# Plot the first images plus their labels 
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
# from https://stackoverflow.com/questions/43178668/record-the-computation-time-for-each-epoch-in-keras-during-model-fit
# Class which enables to store the execution time per epoch
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
def acc_time(time_stamps):
   # accumulate all single time values
    accumulated_time = []
    acc_time = 0
    for time in time_stamps:
        acc_time += time
        accumulated_time.append(acc_time)
    #print(accumulated_time)   
    return accumulated_time
    

In [ ]:
def run_model(model, x_train, y_train, x_test, y_test, class_names):
    print(model.summary())
    print()
    # compile the given model architecture
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # call class for taking time stamps for each epoch
    time_callback_train = TimeHistory()
    
    # fit the model (train the network) and save metrics in variable history
    history = model.fit(x_train, y_train, epochs=50, batch_size=200, callbacks=[time_callback_train])
    
    # store time stamps per epoch in variable
    times_train = small_cnn_time_callback_train.times
    print()
    print("Reported times per epoch: \n ", times_train)
    
    accumulated_time = acc_time(times_train)
    
    # Evaluate model using testing dataset
    test_start_time = time.time()
    test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=200, verbose=2)
    print()
    print()
    print('Test Loss: {} and Test Accuracy: {}'.format(test_loss, test_acc))
    test_end_time = time.time() - test_start_time
    print('Time for Testing Data: ', test_end_time)
    
    
    # show plots
    # Plot training accuracy values vs epochs
    plt.plot(history.history['accuracy'])
    plt.title('Model accuracy vs Epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training loss values vs epochs
    plt.plot(history.history['loss'])
    plt.title('Model loss vs Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training time vs epochs
    plt.plot(accumulated_time, history.history['loss'])
    plt.title('Model Loss vs Time')
    plt.ylabel('Loss')
    plt.xlabel('Time in seconds')
    plt.legend(['Train'], loc='upper left')
    plt.show()
    
    return history

In [ ]:
def conf_mat(model, x_test, y_test):
    # Generate prediction output from model using scaled testing dataset
    # Using keras predict_classes
    predictions = model.predict_classes(x_test, verbose=2)

    # Creating the confusion matrix using sklearn library
    conf_matrix = confusion_matrix(y_test, predictions)
    # Changing confusing matrix output using pandas library
    conf_matrix = pd.DataFrame(conf_matrix)
    conf_matrix
    # Changing index / columns names to class names of output
    conf_matrix.columns = class_names
    conf_matrix.index = class_names
    
    return(conf_matrix)

# Task 1: Fully Connected Neural Network

In [ ]:
# Create the model
FC_model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28), name="FC_Input_Layer"),
    keras.layers.Dense(784, activation='tanh', name="FC_Hidden_Layer_1"),
    keras.layers.Dense(512, activation='sigmoid',name="FC_Hidden_Layer_2"),
    keras.layers.Dense(100, activation='relu', name="FC_Hidden_Layer_3"),
    keras.layers.Dense(10, activation='softmax', name="FC_Output_Layer")],
    name='Fully_Connected_NN')

# Task 2: Small Convolutional Neural Network

In [ ]:
Small_CNN = Sequential()
Small_CNN.add(layers.Conv2D(filters=40, kernel_size=(5,5),strides=5,padding='valid', activation='relu', input_shape=(28, 28, 1)))
Small_CNN.add(layers.MaxPooling2D((2, 2)))
Small_CNN.add(layers.Flatten())
Small_CNN.add(layers.Dense(100, activation='relu'))
Small_CNN.add(layers.Dense(10, activation='softmax'))

# Task 3: Bigger Convolutional Neural Network

In [ ]:
# Built the CNN architecture
bigger_CNN = Sequential(name='Bigger_CNN')
bigger_CNN.add(layers.Conv2D(filters=48, kernel_size=(3,3),strides=1,padding='valid',activation='relu',input_shape=(28,28,1)))
bigger_CNN.add(layers.MaxPooling2D((2,2,)))
bigger_CNN.add(layers.Conv2D(filters=96, kernel_size=(3,3), strides=1, padding='valid', activation='relu'))
bigger_CNN.add(layers.MaxPooling2D((2,2,)))
bigger_CNN.add(layers.Flatten())
bigger_CNN.add(layers.Dense(100, activation='relu'))
bigger_CNN.add(layers.Dense(10, activation='softmax'))

# Run the networks

In [ ]:
# Task 1: Fully Connected Network
run_model(FC_model, x_train_scaled, y_train_cat, x_test_scaled, y_test_cat, class_names)
conf_mat(FC_model, x_test_scaled, y_test)

In [ ]:
# Task 2: Small CNN
run_model(small_cnn, reshaped_x_train_scaled, y_train_cat, reshaped_x_test_scaled, y_test_cat, class_names)
conf_mat(small_cnn, reshaped_x_test_scaled, y_test)

In [ ]:
# Task 3: Bigger CNN
run_model(bigger_CNN, reshaped_x_train_scaled, y_train_cat, reshaped_x_test_scaled, y_test_cat, class_names)
conf_mat(bigger_CNN, reshaped_x_test_scaled, y_test)